# Background

The Mandelbrot set is the set of complex numbers $c$ for which the function $f_{c}(z)=z^{2}+c$ does not diverge when iterated from $z=0$.

Images of the Mandelbrot set are created by sampling a range of complex numbers and for each point determining how the preceeding iterative function behaves. The examples in this notebook use the [Escape time algorithm](https://en.wikipedia.org/wiki/Mandelbrot_set#Escape_time_algorithm), where the colour assigned to each image pixel is determined by how many iterations are required for the function to diverge or not. The algorithm performs an early exit as soon as the function value exceeds a given threshold or horizon. Pixels that do not diverge after the maximum number of iterations are assumed to belong to the Mandelbrot set and are assigned the colour black. Ironically, this means that the interesting images commonly associated with the Mandelbrot set are actually plots of all the points that *don't* belong to the set!

## About the Computational Characteristics
Calculating the members of the Mandelbrot Set is an example of an [embarrassingly parallel](https://en.wikipedia.org/wiki/Embarrassingly_parallel) problem. In this case, each pixel of the image can be calculated without reference to either the inputs or results of any other pixel. The computation is loop-heavy, and does not require any disc IO. Performance tends to be dominated by CPU speed, algorithm details (particularly in the inner loop), and memory access patterns (cache performance). This may be classed as a typical [CPU-bound](https://en.wikipedia.org/wiki/CPU-bound) problem.

# Topics covered
* Python type hinting
* [Google Style](https://google.github.io/styleguide/pyguide.html) pydoc strings
* Passing functions as arguments to other functions
* Plotting with matplotlib
* Just-in-time compilation with numba
* Using Cython
* Vectorisation, and the importance of cache-friendly code
* Profiling inside Jupyter Notebooks
    * `%timeit` to measure average execution time
    * `%prun`

# Imports

In [ ]:
import numpy as np
import numexpr as ne
from matplotlib import pyplot as plt
from matplotlib import colors
from numba import jit
%matplotlib inline
%load_ext Cython

# Simple Version
First, define the function that determines Mandelbrot set membership for a complex number using the Escape time algorithm:

In [ ]:
def mandelbrot_simple(
    z: complex, 
    horizon: float, 
    max_iterations: int) -> int:
    """ Calculate Mandelbrot set membership for a given complex number.
    
    Once the iterative function value exceeds the horizon, the algorithm escapes through an early exit, 
    returning the current number of iterations. If the value does not exceed the horizon by in the maximum 
    number of iterations, then the point is assumed to be a member of the Mandelbrot set, 
    and the special flag value of 0 is returned.
    
    Args:
        z (complex): The complex number to calculate
        horizon (float): The escape horizon.
        max_iterations (int): Maximum number of iterations
        
    Returns:
        The number of iterations required for divergence, or 0 if the max number of iterations was reached.
    """
    c = z
    z = 0.0j
    for n in range(max_iterations):
        # Does the magnitude of z exceed the horizon?
        if np.abs(z) > horizon:
            # Yes, so escape and return the number of iterations so far
            return n
        # No, so perform another iteration
        z = z * z + c
        
    # Did not diverge after max_iterations
    return 0

Now define a function that loops over the sampled complex numbers, calling the mandelbrot_simple function for each number in a 2D array. 

Our first approach therefore uses nested loops to traverse the image pixels in row-major order.

Another feature to note is that the iterative function called at each pixel is passed as an argument. This will allow us to easily reuse this function with different versions of the per-pixel function.

First, let's define some global constants that will allow us to quickly change the defaults used in all implementations.

In [ ]:
default_r_min = -2.0
default_r_max = 0.5
default_i_min = -1.25
default_i_max = 1.25
default_r_samples = 800
default_i_samples = 800
default_horizon = 20000.0
default_max_iterations = 100

In [ ]:
def mandelbrot_set(
    mandelbrot_func: callable = mandelbrot_simple,
    r_min: float = default_r_min,
    r_max: float = default_r_max,
    i_min: float = default_i_min,
    i_max: float = default_i_max,
    r_samples: int = default_r_samples,
    i_samples: int = default_i_samples,
    horizon: float = default_horizon,
    max_iterations: int = default_max_iterations):
    """ Mandelbrot set membership, version 1.
    
    Samples points in the complex plane and for each point calculates whether the 
    point is a member of the Mandelbrot set.
    
    Args:
        mandelbrot_func (callable): The function to call at each sampled point.
        r_min (float): lower bound of the real number interval
        r_max (float): upper bound of the real number interval
        i_min (float): lower bound of the imaginary number interval
        i_max (float): upper bound of the imaginary number interval
        r_samples (int): Number of samples along the real axis
        i_samples (int): Number of samples along the imaginary axis
        horizon (float): The escape horizon.
        max_iterations (int): Maximum number of iterations
        
    Returns:
        A 3-tuple containing the real parts of the sampled complex numbers, 
        the imaginary parts, and the number of iterations before divergence, 
        all in numpy ndarrays.
    """
    # Create a linearly-spaced sample of single-precision points in the complex plane
    real_parts = np.linspace(r_min, r_max, r_samples, dtype=np.float32)
    imaginary_parts = np.linspace(i_min, i_max, i_samples, dtype=np.float32)
    
    # We need somewhere to store the number of iterations at each point. 
    # Lets use a r_samples * i_samples ndarray
    num_iterations = np.zeros((r_samples, i_samples))
    
    # What happens if the imaginary_parts[j]*1j is moved into a ufunc operation outside the loop?
    for i in range(r_samples):
        for j in range(i_samples):
            num_iterations[i, j] = mandelbrot_func(
                real_parts[i] + 1j * imaginary_parts[j],
                horizon,
                max_iterations)
            
    return (real_parts, imaginary_parts, num_iterations)

Before we get to examining the performance, let's make a pretty picture...

# Rendering the Mandelbrot Set
Although the focus of this notebook is on performance characteristics of the Mandelbrot set generation, it would be a shame to not make some pretty fractal images. To do so, we turn to matplotlib, defining a function that can render an inline image from our results tuples. In addition, it displays a colorbar indicating the number of iterations mapped to the colours.

The gamma correction is used to account for the fact that most pixels diverge after just a few iterations, and thus all the colours are bunched into one part of the colour map. You can see the effect of gamma correction in the colour bars by adjusting the gamma value.

In [ ]:
def render(
    results, 
    image_width=10,
    image_height=10,
    r_min=default_r_min,
    r_max=default_r_max,
    i_min=default_i_min,
    i_max=default_i_max,
    dpi=72, 
    cmap='inferno', 
    gamma=0.3):
    
    # unpack the results tuple into x, y, and z.
    # (x, y) are the image coordinates.
    # z is the function value to be mapped to a colour value
    x,y,z = results
    
    num_pixels_x = len(x)
    num_pixels_y = len(y)
    
    # create the matplotlib figure and axes
    fig, ax = plt.subplots(figsize=(image_width, image_height), dpi=dpi)
    x_tick_locations = np.arange(0, num_pixels_x, 3 * dpi)
    x_tick_labels = r_min + (r_max - r_min) * x_tick_locations / num_pixels_x
    plt.xticks(x_tick_locations, x_tick_labels)
    
    y_tick_locations = np.arange(0, num_pixels_y, 3 * dpi)
    y_tick_labels = i_min + (i_max - i_min) * y_tick_locations / num_pixels_y
    plt.yticks(y_tick_locations, y_tick_labels)
          
    # Calculate the power norm used for colour correction
    norm = colors.PowerNorm(gamma)
    plt.imshow(z.T, cmap=cmap, origin='lower', norm=norm, interpolation='bicubic')
    plt.colorbar()

Now that we have a render function, we can calculate an image. We first determine the number of complex points to sample, starting from the desired image dimensions.

In [ ]:
width = 10
height = 10
dpi = 72
num_pixels_x = dpi * width
num_pixels_y = dpi * height

Now we can calculate the set memberships. Since the simple implementation is quite slow, we override the default `max_iterations` with a smaller value.

In [ ]:
results = mandelbrot_set(
    r_samples=num_pixels_x,
    i_samples=num_pixels_y,
    max_iterations=30)

In [ ]:
render(results, image_width=width, image_height=height, dpi=dpi, gamma=0.5)

If you want to explore the visual world of the Mandelbrot set, I suggest waiting until the end of this notebook when a much faster method of calculation is available.

# Tracking our Results
To help visualise the different performance improvements, let's define a class to track and plot our timing results.

In [ ]:
class Results(object):
    def __init__(self, baseline_time, baseline_label, color):
        self.speedups = [1]
        self.labels = [baseline_label]
        self.colors = [color]
        self.baseline_time = baseline_time
        self.times = []
        
         # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.times.append(self.baseline_time.average)
        except:
            # Failing that, older versions have a list of all times
            self.times.append(np.mean(self.baseline_time.all_runs))
        
    def speedup_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, speedup, color in zip(self.labels, ind, self.speedups, self.colors):
            ax.bar(
                index,
                speedup, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Speedup relative to {}'.format(self.labels[0]))
        ax.legend(self.labels)
        plt.show()
        return self
        
    def walltime_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, walltime, color in zip(self.labels, ind, self.times, self.colors):
            ax.bar(
                index,
                walltime, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Walltime')
        ax.legend(self.labels)
        plt.show()
        return self
        
    def add_time(self, time, label, color):
        
        baseline_avg = self.times[0]
        # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.speedups.append(baseline_avg / time.average)
            self.times.append(time.average)
        except:
            # Failing that, older versions have a list of all times
            avg = np.mean(time.all_runs)
            self.speedups.append(baseline_avg / avg)
            self.times.append(avg)
            
        self.labels.append(label)
        self.colors.append(color)
        
        # return a reference to self to allow chaining
        return self

# Exploring the Performance of mandelbrot_set_simple

For comparison, we will use the default parameters from now on. First, lets measure the execution time of `mandelbrot_set_simple`. Note that our simple implementation is so slow that we deliberately limit the number of calls to our function (the `%timeit` default is to make multiple calls and then average the results).

In [ ]:
times_simple = %timeit -n 1 -r 1 -o results = mandelbrot_set()

Create a results object and add our baseline result. This will be used to track our results and calculate the relative speedup. We don't bother with a plot until after gathering the second result.

In [ ]:
results = Results(times_simple, 'simple', 'blue')

## Profiling
We are often advised to profile our code first before making optimisations. So lets try the `%prun` magic that reports function-level profiling information. To make this a bit faster, the number of samples is reduced, but the results should still be representative.

In [ ]:
%prun mandelbrot_set(r_samples=300, i_samples=300)

The clear majority of time is spent in the per-pixel `mandelbrot_simple` function, so lets see if we can improve its performance.

## Improving the Algorithm

Have a look at the `mandelbrot_simple` function. The divergence check compares the magnitude (calculated with `np.abs`) of the current value of `z` with the `horizon` parameter. This is defined as $\lvert x + iy \rvert = \sqrt{x^2 + y^2}$

Square roots can be expensive...

In [ ]:
n = 3 + 4j

In [ ]:
%time np.abs(n)

That doesn't take long, but is there a faster way? One common method to avoid a square root when making a magnitude comparison is to compare the squared magnitude. 

How long does calculating the squared magnitude of a complex number take? The squared magnitude for $x + iy$ is $x^2 + y^2$. We don't have a built-in operator for this function, so it is written in full:

In [ ]:
%time n.real * n.real + n.imag * n.imag

On my computer, that takes approximately 1/5 of the time. Let's see what difference that saving makes when buried in the inner loop that is executed thousands of times:

In [ ]:
def mandelbrot_magsquared(z, horizon, max_iterations):
    c = z
    z = 0.0j
    
    # The horizon is a loop-invariant, therefore it can be calculated just once
    horizon_squared = horizon * horizon
    
    for n in range(max_iterations):
        # Does the magnitude of z exceed the horizon?
        if z.real * z.real + z.imag * z.imag > horizon_squared:
            # Yes, so escape and return the number of iterations so far
            return n
        # No, so perform another iteration
        z = z * z + c
        
    # Did not diverge after max_iterations
    return 0

In [ ]:
times_magsquared = %timeit -n 1 -r 1 -o mandelbrot_set(mandelbrot_func=mandelbrot_magsquared)

In [ ]:
results.add_time(times_magsquared, 'mag squared', 'skyblue').speedup_plot();

That's a little faster.

Notice how the `z.real * z.real + z.imag * z.imag` calculation is similar to the expression that squares z a few lines down? What if we stop using the `complex` type, and handle the real and imaginary parts separately all the way through? This might allow doing some calculations only once and then reusing the results, thus reducing number of calculations for each iteration. Note that the square of a complex number is defined as:

$$(x + iy)^2 = x^2 + 2xiy + (iy)^2$$

Splitting into separate calculations for the real and imaginary parts, and noting that $i^2 = -1$ gives

$$real = x^2 - y^2$$ 

$$imag = 2xy$$

In [ ]:
def mandelbrot_complex_number_split(z, horizon, max_iterations):
    c_real = z.real
    c_imag = z.imag
    z_real = 0.0
    z_imag = 0.0
    
    # The horizon is a loop-invariant, therefore it can be calculated just once
    horizon_squared = horizon * horizon
    
    for n in range(max_iterations):
        # precompute some shared values
        z_real_squared = z_real * z_real
        z_imag_squared = z_imag * z_imag
        # Does the magnitude of z exceed the horizon?
        if z_real_squared + z_imag_squared > horizon_squared:
            # Yes, so escape and return the number of iterations so far
            return n
        # No, so perform another iteration
        # Note that we calculate the new value of z_imag first. Why?
        z_imag = 2 * z_real * z_imag + c_imag
        z_real = z_real_squared - z_imag_squared + c_real
                
    # Did not diverge after max_iterations
    return 0

In [ ]:
times_complex_number_split = %timeit -n 1 -r 1 -o mandelbrot_set(mandelbrot_func=mandelbrot_complex_number_split)

In [ ]:
results.add_time(times_complex_number_split, 'complex split', 'cadetblue').speedup_plot();

That's a little faster again, but the code is getting harder to read. And we are still only getting less than twice the speedup of the original. Is that the best we can hope for with Python? Should we be reaching for C or Fortran? Before going that far, let's look at some other options that stay with Python.

There is actually a serious performance problem with the simple implementation that cannot be solved by optimising the per-pixel function. Can you see what it is?

**Hint:** It's related to loops and data access. Let's look at a NumPy implementation to see if it can shed some light on what is happening.

# NumPy
An internet search reveals lots of ways to rewrite this algorithm for numpy. Here we will look at just one approach while comparing the square-root removal optimisation.

In [ ]:
def mandelbrot_numpy_vectorised_1(c, horizon, max_iterations):
    'Vectorised Numpy implementation with magnitude escape test'
    iterations = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    
    for it in range(max_iterations):
        # check for divergence using the original magnitude test
        notdone = np.less(np.absolute(z), horizon)
        iterations[notdone] = it
        z[notdone] = z[notdone]**2 + c[notdone]
    iterations[iterations == max_iterations - 1] = 0
    return iterations

In [ ]:
def mandelbrot_numpy_vectorised_2(c, horizon, max_iterations):
    'Vectorised Numpy implementation with magnitude-squared escape test'
    iterations = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    
    # The horizon is a loop-invariant, therefore it can be calculated just once
    horizon_squared = horizon * horizon
    
    for it in range(max_iterations):
        # Note that we are incorporating the magnitude-squared optimisation to avoid a square root
        notdone = np.less(z.real*z.real + z.imag*z.imag, horizon_squared)
        iterations[notdone] = it
        z[notdone] = z[notdone]**2 + c[notdone]
    iterations[iterations == max_iterations - 1] = 0
    return iterations

There are a few interesting points to note about this implementation, including:
* the use of boolean indexing to stop calculations on pixels that have already diverged,
* rather than calculating pixels one at a time, all pixels have a single iteration performed at the same time

The `mandelbrot_numpy_vectorised` function takes an entire numpy array of complex numbers instead of a single complex number. This requires a new `mandelbrot_set` wrapper function to call the vectorised calculation functions. We didn't call this wrapper `mandelbrot_set_numpy` as we reuse it with other implementations later on.

In [ ]:
def mandelbrot_set_vectorised(
    mandelbrot_func: callable,
    r_min: float = default_r_min,
    r_max: float = default_r_max,
    i_min: float = default_i_min,
    i_max: float = default_i_max,
    r_samples: int = default_r_samples,
    i_samples: int = default_i_samples,
    horizon: float = default_horizon,
    max_iterations: int = default_max_iterations):
    
    # Create a linearly-spaced sample of single-precision points in the complex plane
    real_parts = np.linspace(r_min, r_max, r_samples, dtype=np.float32)
    imaginary_parts = np.linspace(i_min, i_max, i_samples, dtype=np.float32)
    # Combine the real and imaginary samples into a single 2D array of complex numbers
    c = real_parts + imaginary_parts[:,None]*1j
        
    # calculate set membership
    num_iterations = mandelbrot_func(c, horizon, max_iterations)
            
    # Transpose the num_iterations array so the shape and pixel ordering matches the previous implementations
    return (real_parts, imaginary_parts, num_iterations.T)

In [ ]:
times_numpy_1 = %timeit -o mandelbrot_set_vectorised(mandelbrot_func=mandelbrot_numpy_vectorised_1)

In [ ]:
times_numpy_2 = %timeit -o mandelbrot_set_vectorised(mandelbrot_func=mandelbrot_numpy_vectorised_2)

In [ ]:
results.add_time(times_numpy_1, 'numpy', 'firebrick')
results.add_time(times_numpy_2, 'numpy mag squared', 'tomato')
results.speedup_plot();

That's a much more impressive speedup than our optimisations to the `mandelbrot_simple` function. We can also see that the magnitude squared escape test is still worthwhile.

What is going on here? How is such a performance increase being achieved?

## Memory Access, Cache Misses, and Vectorisation
The simple solution uses numpy arrays to hold the data, but fails to take advantage of the memory layout of that data. Instead, the real and imaginary values for each number are copied one at a time, used to construct a new complex number object which is then passed to the per-pixel function. Copying data when creating the new `Complex` object puts that data into a new memory location, meaning it is probably not already in cache. This causes a cache miss when the data is read, leading to a potential CPU stall while data is loaded from system RAM. On the other hand, the NumPy version is implemented with Universal Functions or ufuncs. These have several charactistics that help performance:
* implemented to take advantage of the contiguous memory layout of Numpy ndarrays, leading to substantially more effective use of cache memory.
* frequently implemented in C, bypassing some additional Python overheads such as bounds checking
* operate directly on the data, avoiding the additional copies of our simple implementation and often avoiding index safety checks
* depending on numpy compile options, they may call into optimised linear algebra libraries like BLAS or Intel MKL for some functions

Since accessing higher cache levels or main RAM is substantially slower than most CPU instructions, cache miss effects can easily dominate performance in loop-heavy CPU-bound code like this function we are exploring here.

Unfortunately, the simple profiling tools (such as `%timeit` and `%prun`) available in notebooks do not allow cache performance profiling.

NumPy is not the end of the story though. NumExpr is another library that can improve performance over NumPy for some data sets and algorithms. Let's examine it now.

# Numexpr

[Numexpr](https://github.com/pydata/numexpr) is a fast numerical expression evaluator for NumPy. It avoids allocating memory for intermediate results, leading to improved cache utilisation when compared to the default NumPy implementations.

However, in practice, Numexpr sometimes performs better and sometimes worse than the corresponding NumPy implementation. According to the developers, it does best on large matrices that don't fit into CPU cache. You can explore the impact of matrix size on the relative performance of Numexpr by adjusting the values for `default_r_samples` and `default_i_samples` before running all cells again. **Note:** if you increase the array dimensions to more than 1000, you probably need to lower `max_iterations` to a small number like 50, otherwise the total execution time will be too high.

Fortunately we require only minor modifications to the NumPy implementation to use Numexpr, so let's see how it does. We only examine the magnitude-squared escape test here. If you want to compare the original magnitude test, try modifying the code yourself.

In [ ]:
def mandelbrot_numexpr(c, horizon, max_iterations):
    iterations = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    
    # The horizon is a loop-invariant, therefore it can be calculated just once
    horizon_squared = horizon * horizon
    
    for it in range(max_iterations):
        notdone = ne.evaluate('z.real*z.real + z.imag*z.imag < horizon_squared')
        iterations[notdone] = it
        z = ne.evaluate('where(notdone, z**2 + c, z)')
    iterations[iterations == max_iterations - 1] = 0
    return iterations

In [ ]:
times_numexpr = %timeit -o mandelbrot_set_vectorised(mandelbrot_func=mandelbrot_numexpr)

In [ ]:
results.add_time(times_numexpr, 'numexpr', 'orange').speedup_plot();

# Numba
So far we are doing really well. On my test machine, the Numexpr version is giving ~75x speedup.

But we are not out of options yet. [Numba](http://numba.pydata.org/) is a new library that uses [LLVM](http://llvm.org/) to perform just-in-time (JIT) compilation to native code. It is particularly well suited to array computations. It integrates well with the SciPy stack and can target both CPU and GPU hardware. Let's see how easy it is to compile our Mandelbrot functions. Essentially all we need to do in this case is add the `@jit` decorator to our original simple implementation. Yes, the one with the slow loops and poor memory access patterns!

Note that the per-pixel function is hard-coded. Numba will work if this function is passed as an argument but performance degrades drastically (try it and see). Why do you think this happens?

In [ ]:
@jit
def mandelbrot_numba(z, horizon, max_iterations):
    c = z
    z = 0.0j
    for n in range(max_iterations):
        # Does the magnitude of z exceed the horizon?
        if abs(z) > horizon:
            # Yes, so escape and return the number of iterations so far
            return n
        # No, so perform another iteration
        z = z * z + c
        
    # Did not diverge after max_iterations
    return 0

In [ ]:
@jit
def mandelbrot_set_numba(
    r_min: float = default_r_min,
    r_max: float = default_r_max,
    i_min: float = default_i_min,
    i_max: float = default_i_max,
    r_samples: int = default_r_samples,
    i_samples: int = default_i_samples,
    horizon: float = default_horizon,
    max_iterations: int = default_max_iterations):
    
    # Create a linearly-spaced sample of single-precision points in the complex plane
    real_parts = np.linspace(r_min, r_max, r_samples, dtype=np.float32)
    imaginary_parts = np.linspace(i_min, i_max, i_samples, dtype=np.float32)
    num_iterations = np.zeros((r_samples, i_samples))
    
    for i in range(r_samples):
        for j in range(i_samples):
            num_iterations[i, j] = mandelbrot_numba(
                real_parts[i] + 1j * imaginary_parts[j],
                horizon,
                max_iterations)
            
    return (real_parts, imaginary_parts, num_iterations)

In [ ]:
times_numba = %timeit -o mandelbrot_set_numba()

In [ ]:
results.add_time(times_numba, 'numba', 'gold').speedup_plot();

# Cython

The [Cython](http://cython.org/) implementation fairly closely follows the simple and Numba versions, however there are several  points worth noting:
* The `%%cython` cell magic, telling Jupyter that this is a Cython code cell.
* The code requires the most changes. Unlike the Numba code, you can't just comment out a single line and revert to pure Python.
* Explicit types are used for **all** variables. Cython allows you to use implicit types, but it defaults to an undifferentiated Python object. This nearly always results in slower code.
* Use of the `@cython.boundscheck(False)` decorator to turn off array bounds-checking for the entire function. This provides a modest speed improvement.
* Use of the `@cython.wraparound(False)` decorator to turn off negative array index wrapping for entire function. This provides a modest speed improvement. This can only be used if you never use negative indicies in the code.
* Complex numbers are explicitly split into separate real and imaginary parts, this gave much better performance.
* The per-pixel function is declared inline with an explicit return value. This gives a huge performance improvement.
* The three ndarrays have explicit dimensions and types specified in the Cython type definition. For arrays that will only be accessed by simple integer indicies this allows Cython to use pointer arithmetic for very fast data access. You can't use this approach if more complex slicing is used.
* Although Cython does allow default function arguments, it only appeared to work with literal values. I couldn't get it to work with the approach used on all the other versions.
* This code took a lot longer to write and tune when compared to any of the other versions.

Try changing some of the code options and re-running the timing test to explore the impact.

In [ ]:
%%cython
import numpy as np
cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline long mandelbrot_cython(float c_real, float c_imag, float horizon, long max_iterations):
    cdef float z_real = 0.0, z_real_sq = 0.0
    cdef float z_imag = 0.0, z_imag_sq = 0.0
    cdef long n = 0
    cdef float horizon_sq = horizon * horizon
    for n in range(max_iterations):
        # precompute some shared values
        z_real_sq = z_real * z_real
        z_imag_sq = z_imag * z_imag
        # Escape check
        if z_real_sq + z_imag_sq > horizon_sq:
            return n 
        z_imag = 2 * z_real * z_imag + c_imag
        z_real = z_real_sq - z_imag_sq + c_real
        
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
def mandelbrot_set_cython(
    float r_min,
    float r_max,
    float i_min,
    float i_max,
    long r_samples,
    long i_samples,
    float horizon,
    long max_iterations):
    
    # Create a linearly-spaced sample of single-precision points in the complex plane
    cdef np.ndarray[np.float32_t, ndim=1] real_parts = np.linspace(r_min, r_max, r_samples, dtype=np.float32)
    cdef np.ndarray[np.float32_t, ndim=1] imaginary_parts = np.linspace(i_min, i_max, i_samples, dtype=np.float32)
    cdef np.ndarray[np.int32_t, ndim=2] num_iterations = np.zeros((r_samples, i_samples), dtype=np.int32)
        
    for i in range(r_samples):
        for j in range(i_samples):
            num_iterations[i, j] = mandelbrot_cython(
                real_parts[i],
                imaginary_parts[j],
                horizon,
                max_iterations)
            
    return (real_parts, imaginary_parts, num_iterations)

In [ ]:
times_cython = %timeit -o mandelbrot_set_cython(default_r_min, default_r_max,default_i_min, default_i_max,default_r_samples, default_i_samples,default_horizon, default_max_iterations)

In [ ]:
results.add_time(times_cython, 'Cython', 'sienna').speedup_plot();

# Conclusions
We have explored a lot of options for accelerating Python code. But what is the right method for your particular application? Unfortunately, there doesn't appear to be an answer that is right for all situations. If I had chosen a different problem, or even different parameters for this algorithm then the relative results may vary substantially.

Vectorised NumPy code will provide performance that is fast enough in most scientific applications, with the added advantage of good portability. Numba and Cython can be hard to install if you are not using Anaconda, so they may make code hard to share with others or to use in a library.

If the default NumPy performance is not sufficient, then NumExpr is particularly easy to test, as the code is quite similar to NumPy ufunc-based implementations. However the relative performance of NumExpr seems particularly sensitive to interactions between the array size and system cache. On my desktop, NumExpr nearly always outperforms NumPy but on Pearcey NumExpr does worse unless the array size is very large. But if I reduce the image size to 300x300 or less, the NumExpr version becomes slower than the Numpy version.

Numba can frequently outperform NumPy on loop-centric functions, but it imposes a lot more limitations on the Python language features that can be used. You may also need to rewrite your code to use explicit loops. And as we have seen, that is not usually a good idea. Another use of Numba that was not explored here is using it to implement your own custom NumPy ufuncs. This feature can be particularly powerful.

Cython can often provide the most performance benefit, but it also requires the most additional work. It also leads to code that bears the least resemblance to Python. However, when performance is critical, this may be a simpler option that writing directly in C, C++, or Fortran.

It is worth noting that although the relative performance of NumExpr is quite variable in response to the array size and the number of iterations, Numba and Cython consistently outperform all other implementations of this algorithm.

As always, the performance improvements should be guided by measurement and profiling. But what about the profiling we did here? Didn't it point to the inner function as the main bottle neck? Yes, but it is important to interpret the profiling results in light of what we know about the code. In this case, optimising both the outer and inner functions together allowed optimisations of the loop code as well as the inner per-pixel function. You can explore this with the Numba and Cython implementations by passing the optimised inner function to the original `mandelbrot_set` outer function.

## Diminishing Returns: Why *Fast Enough* is often *Fast Enough*
While the speedup plot we have been using is useful for understanding the relative performance of different optimisation approaches, it can give a misleading impression of the practical effect on job runtimes. To examine this, lets first look at the plot of walltimes from our results.

In [ ]:
results.walltime_plot();

Clearly, moving to a NumPy implementation led to a dramatic reduction in total execution time (the walltime). But despite all our additional efforts, and the impressive speedup numbers, each successive optimisation only reduced the execution time by a relatively small amount. Sometimes *fast enough is fast enough*.

Lets illustrate this further by scaling our timing results to see how long each job would take if we assume:
* That the original `simple` version actually took 12 hours to complete.
* The relative speedup obtained was the same as our current results.

In [ ]:
assumed_walltime_minutes_slow_code = 12 * 60
scale = assumed_walltime_minutes_slow_code / results.times[0]
new_times = [t * scale for t in results.times]
plt.scatter(x=range(len(new_times)), y=new_times)
max_len = max((len(l) for l in results.labels))
for t, l in zip(new_times, results.labels):
    padding = ' ' * (max_len - len(l) + 1)
    print('{0}:{2}{1:>6.2f} minutes'.format(l, t, padding))

So in our scenario, switching to NumPy would reduce the run time from 12 hours to around 15 minutes (or even less with the magnitude squared optimisation). As impressive as the additional speedup from Cython is, absolute reduction in run time is smaller than the difference between the simple and Numpy versions. Whether those extra minutes are worth the additional programming effort is an open question, with a different answer for every application.

# Exploring the Image
Now that we have some decent performance, it is possible to explore ...

* Changing the [colormap](https://matplotlib.org/examples/color/colormaps_reference.html)
* Changing the gamma value
* Changing the image dimensions (`dpi`, `width`, `height`)
* Changing the region of the complex plane that is calculated (`r_min`, `r_max`, `i_min`, and `i_max`). Note that for the Mandelbrot set, all the interesting things happen within a radius of 2 around the origin.

To get started, let's try looking at $(-0.749, 0.065)$ to $(-0.748, 0.066)$

In [ ]:
width = 12
height = 12
dpi = 72
num_pixels_x = dpi * width
num_pixels_y = dpi * height

image_results = mandelbrot_set_numba(
    r_samples=num_pixels_x,
    i_samples=num_pixels_y,
    r_min=-0.749, r_max=-0.748,
    i_min=0.065, i_max=0.066,
    horizon=default_horizon, max_iterations=3000)

render(
    image_results, 
    image_width=width, image_height=height, 
    r_min=0.4, r_max=0.7,
    i_min=0.3, i_max=0.6,
    dpi=dpi, gamma=0.8,
    cmap='rainbow')

# Easter Egg: ASCII Rendering

In [ ]:
def Render_ASCII(iterations):
    charset = ' .oXM@jawrpzgOQEPGJ'
    width, height = iterations.shape
    wrap = len(charset) - 1
    for y in range(height):
        row = [charset[int(i % wrap)] for i in iterations[...,y]]
        print(''.join(row))

image_results = mandelbrot_set_numba(r_samples=100, i_samples=40)
Render_ASCII(image_results[2])

# References
* [NumPy](http://www.numpy.org/)
* [NumExpr](https://github.com/pydata/numexpr/wiki/Numexpr-Users-Guide)
* [Numba](http://numba.pydata.org)
* [Cython](http://cython.org/)
* [Mandelbrot Set](https://en.wikipedia.org/wiki/Mandelbrot_set)
* [Matplotlib Showcase](http://matplotlib.org/examples/showcase/mandelbrot.html)
* Jean Francois Puget [How To Quickly Compute The Mandelbrot Set In Python](https://www.ibm.com/developerworks/community/blogs/jfp/entry/How_To_Compute_Mandelbrodt_Set_Quickly?lang=en)
* Jean Francois Puget [My Christmas Gift](https://www.ibm.com/developerworks/community/blogs/jfp/entry/My_Christmas_Gift?lang=en)
